# Quick start

In [ ]:
import numpy as np
import networkx as nx
import math
import time
import pickle as pkl
import scipy

import sys
sys.path.insert(0, '/home/jaeger/B/Python')
sys.path.insert(0, '/home/jaeger/B/Python/AHK')
import utils

from AHK.ahk import AHK_graphon
from AHK.ahk_generators import data_colors,ahk_sbm,sample_data


from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap


In [ ]:
### Define an AHK model representing a stochastic block model:

colprobs = np.array([0.2,0.3,0.5])
edgeprobs = np.array([[0.6,0.1,0.1],[0.1,0.9,0.1],[0.1,0.1,0.8]])

sbm=ahk_sbm(colprobs,edgeprobs,False)

In [ ]:
### Sample random graphs from the model

minnodes=8
maxnodes=30
samplesize=10

sample=sample_data(sbm,samplesize,minnodes,maxnodes)

### these sampled graphs are now instances of utils.World
### They are converted to networkx graphs for plotting

for g in sample:
    gnx=g.to_nx()
    nodecols=(list(gnx.nodes()[j]['features'] for j in gnx.nodes()))
    nx.draw_networkx(gnx,node_color=nodecols,cmap='Set3')
    plt.show()


## Learning

In [ ]:
# Selecting the training data. Must be a list of graphs in the utils.World type.

traindata=sample

# Initializing a model. This requires to define the histogram partitioning of the [0,1] interval
# that the model uses
# The first argument in the constructor is the signature of the graphs we are dealing with, i.e.
# a specification of the attributes and relations


binbounds=utils.uni_bins(3)
model_learned=AHK_graphon(traindata[0].sig,binbounds)

# Defining settings:
settings={}
settings['num_pi_b']=50
settings['batchsize']=10
settings['soft']=0.01
settings['numepochs']=30
settings['learn_bins']=False
settings['early_stop']=5
settings['with_trace']=False

#Adam params:
settings['ad_alpha']=0.01
settings['ad_beta1']=0.99
settings['ad_beta2']=0.9
settings['ad_epsilon']=10e-8

settings['method']="adam"


In [ ]:
# Random initialization and learning
model_learned.rand_init()
best,loglik,trace=model_learned.learn(settings,traindata)